In [6]:
import pandas as pd

train = pd.read_csv("train.csv")

train['Age']=train['Age'].fillna(train['Age'].mean())

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = pd.get_dummies(train[features], drop_first=True).values
y = train['Survived'].values
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [7]:
#n_estimators = decision tree 몇 개로 앙상블 할건지 결정
#max_depth = 각 decision tree의 최대 깊이
#max_feautures = random feature selection 수
#max_samples = bootstrap sampling에서 각 sample 수
from sklearn.tree import DecisionTreeClassifier
import numpy as np

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, max_features='sqrt', max_samples=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.max_samples = max_samples
        self.estimators = []
        
        for _ in range(n_estimators):
            dt = DecisionTreeClassifier(max_depth=self.max_depth, max_features=self.max_features)
            self.estimators.append(dt)
    
    def fit(self, X, y):
        for i in range(self.n_estimators):
            X_sample, y_sample = self.sample(X, y)
            self.estimators[i].fit(X_sample, y_sample)
            
    def sample(self, X, y):
        if self.max_samples is None:
            n_samples = X.shape[0]  
        else:
            n_samples = min(self.max_samples, X.shape[0])

        indices = np.random.choice(X.shape[0], n_samples, replace=True) # replace = 복원 추출 유무
        return X[indices], y[indices]
    
    def predict(self, X):
        all_predictions = np.zeros((self.n_estimators, X.shape[0]), dtype=np.int64)
        for i in range(self.n_estimators):
            all_predictions[i] = self.estimators[i].predict(X)
        
        # predictions = np.zeros(X.shape[0], np.int64)
        # for i in range(X.shape[0]):
            # predictions[i] = np.bincount(all_predictions[:, i]).argmax()
        return np.array([np.bincount(all_predictions[:, i]).argmax() for i in range(X.shape[0])])
        

In [8]:
rf = RandomForestClassifier(n_estimators=50, max_depth=10)
rf.fit(X, y)

y_pred = rf.predict(X)
print("Training Accuarcy:", (y_pred == y).mean())

Training Accuarcy: 0.9337822671156004


In [9]:
import numpy as np

data = np.array([11, 22, 33, 44, 55, 66, 77, 88, 99])

data[[3, 1, 2]]

# 0 ~ 10 사이의 숫자 중 5개를 중복을 허용하며 뽑음
# indices = np.random.choice(10, 5, replace=True)
# print(indices)
# print(data[indices])

array([44, 22, 33])

In [10]:
n_estimators = 3
X_shape_0 = 10

np.zeros((n_estimators, X_shape_0), dtype=np.int64)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [11]:
#min_samples_split = 해당 수치보다 낮아지면 더 이상 쪼개지지 않음
#oob_score = 원본 데이터에서 sample 되지 않은 데이터들(out of back data)로 정확도를 측정
from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=20,
#                             oob_score=True, max_features=2) 
# max_features = 데이터 중 두개만 골라서 gini impurity 계산해서 낮은거 뽑아감
rf = RandomForestClassifier(n_estimators=50, max_depth=10, max_features='sqrt', oob_score=True, max_samples=0.3,min_samples_split=10)
rf.fit(X, y)
y_pred = rf.predict(X)
print("Training Accuracy:", (y_pred == y).mean())
print("OOB Score:", rf.oob_score_)

Training Accuracy: 0.856341189674523
OOB Score: 0.8260381593714927


In [12]:
import pandas as pd

test = pd.read_csv("test.csv")

test['Age']=test['Age'].fillna(test['Age'].mean())

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_test = pd.get_dummies(test[features], drop_first=True).values
y_test = rf.predict(X_test)

In [13]:
with open("rf_result.csv", "w") as f:
    f.write("PassengerId,Survived\n")
    for pid, survived in zip(test['PassengerId'].values, y_test):
        f.write(f"{pid},{survived}\n")